In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *
from os.path import abspath

spark = SparkSession\
        .builder\
        .appName("pyspark-notebook")\
        .master("spark://spark-master:7077")\
        .config("spark.executor.memory", "512m")\
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
        .enableHiveSupport()\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

## 3. The Data

In [6]:
data = spark.read.csv(path="emp.csv", sep=",", header=True)

In [7]:
data.count()

50

In [10]:
data.describe()

DataFrame[summary: string, EMPLOYEE_ID: string, FIRST_NAME: string, LAST_NAME: string, EMAIL: string, PHONE_NUMBER: string, HIRE_DATE: string, JOB_ID: string, SALARY: string, COMMISSION_PCT: string, MANAGER_ID: string, DEPARTMENT_ID: string]

In [12]:
print(data.head(3))

[Row(EMPLOYEE_ID='198', FIRST_NAME='Donald', LAST_NAME='OConnell', EMAIL='DOCONNEL', PHONE_NUMBER='650.507.9833', HIRE_DATE='21-JUN-07', JOB_ID='SH_CLERK', SALARY='2600', COMMISSION_PCT=' - ', MANAGER_ID='124', DEPARTMENT_ID='50'), Row(EMPLOYEE_ID='199', FIRST_NAME='Douglas', LAST_NAME='Grant', EMAIL='DGRANT', PHONE_NUMBER='650.507.9844', HIRE_DATE='13-JAN-08', JOB_ID='SH_CLERK', SALARY='2600', COMMISSION_PCT=' - ', MANAGER_ID='124', DEPARTMENT_ID='50'), Row(EMPLOYEE_ID='200', FIRST_NAME='Jennifer', LAST_NAME='Whalen', EMAIL='JWHALEN', PHONE_NUMBER='515.123.4444', HIRE_DATE='17-SEP-03', JOB_ID='AD_ASST', SALARY='4400', COMMISSION_PCT=' - ', MANAGER_ID='101', DEPARTMENT_ID='10')]


In [17]:
df1 = data.select(["EMPLOYEE_ID", "PHONE_NUMBER"])

In [16]:
df1.filter(df1['EMPLOYEE_ID'] == '198')

DataFrame[EMPLOYEE_ID: string, PHONE_NUMBER: string]

In [11]:
df1 = df1.join(other=cols_description, on=['column1'], how='left_anti')

In [12]:
df1 = df1.\
               withColumnRenamed("column1", 'c1').\
               withColumnRenamed("column2", "c2").\
               withColumnRenamed("column3", "c3")

In [18]:
df1.write.saveAsTable('hive_table')

In [19]:
ndf= spark.sql("""
select * from hive_table
""")


In [20]:
ndf.show(10)

+-----------+------------+
|EMPLOYEE_ID|PHONE_NUMBER|
+-----------+------------+
|        198|650.507.9833|
|        199|650.507.9844|
|        200|515.123.4444|
|        201|515.123.5555|
|        202|603.123.6666|
|        203|515.123.7777|
|        204|515.123.8888|
|        205|515.123.8080|
|        206|515.123.8181|
|        100|515.123.4567|
+-----------+------------+
only showing top 10 rows

